# Práctica de estructuras

Éste es el código que se utilizará para el desarrollo de la práctica de determinación de estructuras mediante lidar. Para ello el/la alumno/a solo tendrá que modificar las sentencias que se encuentran entre:

#------------------------------------------------------------------------------------------------------------------------------

         Modifique SOLO aquí dentro
#------------------------------------------------------------------------------------------------------------------------------

Importante:
- Un error recurrente puede ser que las rutas se especifiquen con '\\' como separador entre carpetas, mientras que el lenguaje Python utiliza '\\\\' (doble). 
- También se recomienda correr una a una las celdas, y no correr de nuevo una celda mientras se está ejecutando (el software puede quedarse colgado). Si mientras una celda se está ejecutando, se realiza algún cambio en el código de esa celda y queremos volver a ejecutarla, deberemos esperar a que termine o reiniciar los kernels.
- Si lo desea, puede jugar con el resto del código.

**IMPORTACIÓN DE LIBRERÍAS Y ESPECIFICACIÓN DE RUTAS**

En primer lugar **importamos las librerias** necesarias. En el caso de que no se encuentre alguno de los módulos al correr esta celda tendremos que instalar dicho módulo en Anaconda Navigator -> Environments o a través de Anaconda Prompt.

In [ ]:
import os
from pathlib import Path

import numpy as np #For general mathematical calculations
import matplotlib.pyplot as plt #For plotting
from scipy.signal import savgol_filter #Allows to smooth a profile through the Savitzky-Golay filter

#For working with lidar products (developed by the Atmospheric Physics Group of the UGR, GFAT)
from gfatpy.lidar.preprocessing import preprocess
from gfatpy.lidar.plot.quicklook import date_quicklook
from gfatpy.utils.plot import font_axes

También, antes de comenzar con el tratamiento de los ficheros, definimos la ruta en la que se encuentran los datos y en la que queremos guardar las gráficas de resultados.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
databasepath = Path(r'G:\Mi unidad\03. Docencia\UGR22-23\GEOMET\MTMAA\Practica_lidar_estructuras\data\1a')  #Database folder path, until '\1a' (included). Example: 'C:\\Users\\user\\Desktop\\data\\1a'
outputpath =   Path(r'C:\Users\Usuario\Downloads\geomet_output')  #Output folder path. Example: ''C:\\Users\\user\\Desktop\\output''
#------------------------------------------------------------------------------------------------------------------------------

databasepath.is_dir(), outputpath.is_dir() #Comprueba que ambas carpetas existen.



**SERIES TEMPORALES DE SEÑAL CORREGIDA DE RANGO CON RESOLUCIÓN VERTICAL**

A continuación, **se representa la serie temporal con resolución vertical (coloquialmente 'quicklooks')** de los datos lidar medidos. En nuestro caso, representamos el valor de la RCS (range corrected signal) medida en un canal concreto (indicado en el título de la gráfica). Los diferentes canales son:
- 355xta (señal total en 355 nm)
- 532xca; 532xda; 532xpa; 532xpp (señales discriminando la polarización en 532 nm, que se utilizarán en otra práctica)
- 532xta (señal total en 532 nm) **Nosotros trabajaremos con la componente total del canal 532 nm en esta práctica** 
- 1064xta (señal total en 1064 nm)
   
La función date_quicklook() representará los quicklooks de los canales seleccionados para un periodo determinado por `ini_date` y `end_date`. En nuestro caso trabajamos con un solo día por lo que `end_date` = `ini_date`. Los quicklooks se guardarán automáticamente en la carpeta especificada en qlpath, con el formato que se utiliza en la base de datos del grupo de Física de la Atmosfera de la UGR. El proceso toma un tiempo y algunos de los comentarios aparecen en francés. Los comentarios que se muestran por pantalla en esta pestaña no son relevantes (a no ser que sea un error que interrumpa el flujo).

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
qlpath = outputpath / 'quicklooks' #Output quicklook folder path. It is suggested to mantain the proposed path
ini_date = '20200520' #Data for which quicklooks will be obtained. Format yyyymmdd
end_date = ini_date
#------------------------------------------------------------------------------------------------------------------------------
channels = ["532xpa"]
date_quicklook(
    ini_date,
    channels2plot=channels,
    path1a=databasepath,
    figpath=qlpath,
    colorbar_range=(0,5e6),
)

**PERFIL PROMEDIO DE SEÑAL CORREGIDA DE RANGO**

Ahora indicaremos dónde se localizan los ficheros de las medidas realizadas por el lidar. Como se ha explicado en la sesión teórica, siempre que se desea obtener información atmosférica mediante lidar es necesario realizar dos tipos de medidas: la medida atmosférica y la medida de corriente oscura (empleada para realizar correcciones instrumentales). Es necesario especificar ambas para poder procesar correctamente la señal lidar y obtener información atmosférica, procedimiento conocido como **preprocesado**. El archivo de medida atmosférica se nombra como 'mhc_1a_ **Prs**_ YYYYMMDD.nc' y el archivo de corriente oscura como 'mhc_1a_ **Pdc**_ YYYYMMDD_hhmm.nc'. (Nota: Como curiosidad, la función date_quicklook() en la sección anterior tiene implementada internamente la corrección de la señal, por lo que no fue necesario indicar por separado la localización del fichero de la medida y el de corriente oscura para la generación de los quicklooks).

También se indicará el intervalo temporal en el que queremos calcular señal corregida de rango. Los intervalos más interesantes en cada caso serán seleccionados por el/la alumno/a en función de cada escenario atmosférico observado en los quicklooks. Se puede realizar el preprocesado para todas las señales lidar medidas en un día completo, pero debe tener en cuenta que cuanto mayor sea el intervalo temporal seleccionado mayor será el tiempo de ejecución requido por la función. **Se recomienda emplear intervalos temporales de 1 o 2 horas**. 

Una vez especificados los parámetros necesarios, se aplica la **función preprocessing()** que nos corrige la señal lidar medida, obteniendo los productos con los que se puede trabajar.

In [ ]:
#------------------------------------------------------------------------------------------------------------------------------
filepath = databasepath / '2020' / '05' / '20' / 'mhc_1a_Prs_rs_xf_20200520.nc'   #Lidar product path. Example : 'C:\\Users\\user\\Desktop\\data\\1a\\2020\\01\\01\\mhc_1a_Prs_rs_xf_20200101.nc' 
dcfilepath = databasepath / '2020' / '05' / '20' / 'mhc_1a_Pdc_rs_xf_20200520_0821.nc' #Lidar dark current product path
ini_date = '20200520T180000'   #Initial date and time. Format 'yyyymmddThhmmss', where T represents literally the letter T. Example: '20200101T200000'
end_date = '20200520T183000'   #Same format than ini_date
#------------------------------------------------------------------------------------------------------------------------------

if os.path.exists(filepath) and os.path.exists(dcfilepath):
    channels = ["532xpa"]
    data = preprocess(filepath, dc_fl=dcfilepath, channels=channels, ini_date=ini_date, end_date=end_date)
else:
    print("One (or both) of the given files can not be found.")

La siguiente celda permite **representar la señal corregida de rango del canal de estudio, promediando en el intervalo temporal indicado** (en función del interés de la situación atmosférica observada en los quicklooks). Los parámetros sobre el rango vertical pueden modificarse en función del interés del alumno/a. 

Debido a que las señales lidar pueden ser ruidosas, se puede aplicar un suavizado espacial para mejorar la razón señal-ruido (SNR). Para realizar el suavizado del perfil es necesario indicar el número de puntos empleados para el suavizado. Este parámetro será determinado por el/la alumno/a, en función de los resultados que se encuentren en los métodos que se aplican más adelante.

In [ ]:
%matplotlib notebook 
#------------------------------------------------------------------------------------------------------------------------------
ini_date = '20200520T180000'    #Same format as before
end_date = '20200520T183000'    #Same format as before
ini_range = 0.       #Bottom value of the vertical range (height) in meters
end_range = 10000.      #Top value of the vertical range (height) in meters
points2smooth = 15   #Points to use in the smooth of the profile
#------------------------------------------------------------------------------------------------------------------------------
data['rcs_532xpa'] = data.signal_532xpa*data.range**2
sliced_rcs532_smoothed = savgol_filter(data['rcs_532xpa'].sel(time=slice(ini_date,end_date),range=slice(ini_range,end_range)).mean('time').values, points2smooth, 1)
sliced_range = data.coords['range'].values[0:len(sliced_rcs532_smoothed)]

#Plotting
fig, ax1 = plt.subplots(1, 1, figsize = (5, 10))
ax1.set_title(f"Granada, {ini_date[6:8]}/{ini_date[4:6]}/{ini_date[0:4]}\n {ini_date[9:17]} - {end_date[9:17]} UTC")
ax1.set(xlabel = "RCS$_{532}$ [a.u.]", ylabel = "Height [m agl]")
ax1.plot(data['rcs_532xpa'].sel(time=slice(ini_date,end_date),range=slice(ini_range,end_range)).mean('time'),sliced_range/1000., color = 'coral') #Allows to observe the original (unsmoothed) profile
ax1.plot(sliced_rcs532_smoothed,sliced_range/1000.)
ax1.legend(('original', 'smoothed'), scatterpoints=1, loc='upper right', ncol=1, fontsize=10.5) #Represents the legend. Only used in the case that the original profiles is represented
ax1.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax1.xaxis.major.formatter._useMathText = True
ax1.set_ylabel('Range, km agl')
ax1.set_xlim(left = 0)
ax1.set_ylim(bottom = 0)
font_axes(ax1,12)

#The next line allows to save the figure. It will be executed right after the graphic is finished, so if you want to use the
#tools of ""%matplotlib notebook" (e.g. zoom or shift the profile) before saving the figure, you have to use the next cell, 
#where the save line is independent. You can use this technique in all the upcoming cases.
plt.tight_layout()
plt.savefig(outputpath / f"rcs_{ini_date}_{end_date}.png") 

En caso de querer guardar el gráfico tras usar "%matplotlib notebook" use la siguiente celda. 

In [ ]:
#Use this cell in case you want to save a graphic after using the tools of "%matplotlib notebook". You can change the output 
#name of the figure, and you can use this same cell for all the upcoming graphics.
#plt.savefig(outputpath + "\\" + ini_date.replace(":",'')[:-2] + "_" + end_date.replace(":",'')[:-2] + "_RCS.png") 

**PERFIL DE PRIMERA DERIVADA DE SEÑAL CORREGIDA DE RANGO**

El módulo siguiente permitirá **realizar el método de la primera derivada de RCS**, que permite estimar las alturas de la capa límite atmosférica y capas desacopladas. Para su comodidad, las últimas líneas permiten representar líneas horizontales que le permitirán marcar determinadas alturas relevantes.

In [ ]:
%matplotlib notebook 
#------------------------------------------------------------------------------------------------------------------------------
#This four variables allow to zoom the graphic based on the previously observed profile. Some default values are given as a hint
x_min = None  #Bottom value of the RCS first derivative in arbitrary units
x_max = None #Top value of the RCS first derivative in arbitrary units
y_min = None #Bottom value of the vertical range (height) in meters
y_max = None #Top value of the vertical range (height) in meters
#------------------------------------------------------------------------------------------------------------------------------

f_deriv_smoothed = np.gradient(sliced_rcs532_smoothed) #Performs the derivative

#Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 11), sharey = True)
fig.suptitle(f"\n\nGranada, {ini_date[6:8]}/{ini_date[4:6]}/{ini_date[0:4]}\n{ini_date[9:17]} - {end_date[9:17]} UTC")
ax1.set(xlabel = "RCS$_{532}$ [a.u.]", ylabel = "Range [km agl]")
ax1.plot(sliced_rcs532_smoothed,sliced_range/1000.)
ax1.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax1.xaxis.major.formatter._useMathText = True
ax1.set_xlim(left = 0)
ax2.set(xlabel = r"$\frac{dRCS_{532}}{dz}$ [a.u.]")
ax2.plot(np.gradient(data['rcs_532xpa'].sel(time=slice(ini_date,end_date),range=slice(ini_range,end_range)).mean('time')), sliced_range/1000., color = 'coral') #Plots the profile with the original (unsmoothed) products
ax2.plot(f_deriv_smoothed, sliced_range/1000.)
ax2.legend(('original', 'smoothed'), scatterpoints=1, loc='upper right', ncol=1, fontsize=10.5) #Represents the legend. Use only in case the original profile is represented
ax2.axvline(x = 0, color = "k")
if x_min is not None and x_max is not None:
    ax2.set(xlim = (x_min,x_max))
if y_min is not None and y_max is not None:
    ax2.set(ylim = (y_min,y_max))
ax2.ticklabel_format(axis = "x", style = "sci", scilimits = (0,0))
ax2.xaxis.major.formatter._useMathText = True
for ax_ in [ax1, ax2]:
    font_axes(ax_,12)
#Optional. You can represent horizontal lines (as much as needed) to indicate the method height estimation for each layer. If 
#you decided not to use them, you can directly delete the lines or comment them (#__ o """__"""). To use them, simply write
#the height estimated value in height_n.


height_1 = 1. #km
ax1.axhline(y = height_1, color = "r")
ax2.axhline(y = height_1, color = "r")

"""
height_2 = 1.5 #km
ax1.axhline(y = height_2, color = "b")
ax2.axhline(y = height_2, color = "b")

height_3 = 1.2 #km
ax1.axhline(y = height_3, color = "b")
ax2.axhline(y = height_3, color = "b")

height_4 = 1.3 #km
ax1.axhline(y = height_4, color = "g")
ax2.axhline(y = height_4, color = "g")

height_5 = 1.4 #km
ax1.axhline(y = height_5, color = "g")
ax2.axhline(y = height_5, color = "g")
"""

plt.tight_layout()
plt.savefig(outputpath / f"drcs_{ini_date}_{end_date}.png") 

**PERFIL DE PRIMERA DERIVADA LOGARÍTIMICA DE SEÑAL CORREGIDA DE RANGO**

El módulo siguiente permitirá **realizar el método de la primera derivada del logaritmo de RCS**, que permite estimar las alturas de la capa límite atmosférica y capas desacopladas.

In [ ]:
%matplotlib notebook 
#------------------------------------------------------------------------------------------------------------------------------
#This four variables allow to zoom the graphic based on the previously observed profile. Some default values are given as a hint
x_min = None #Bottom value of the RCS first log derivative in arbitrary units
x_max = None #Top value of the RCS first log derivative in arbitrary units
y_min = None #Bottom value of the vertical range (height) in meters
y_max = None #Top value of the vertical range (height) in meters
#------------------------------------------------------------------------------------------------------------------------------

log_smoothed = np.log(sliced_rcs532_smoothed)
f_deriv_log_smoothed = np.gradient(log_smoothed)

#Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (11, 12), sharey = True)
fig.suptitle(f"\n\nGranada, {ini_date[6:8]}/{ini_date[4:6]}/{ini_date[0:4]}\n{ini_date[9:17]} - {end_date[9:17]} UTC")
ax1.set(xlabel = "RCS$_{532}$ [a.u.]", ylabel = "Range [km agl]")
ax1.plot(sliced_rcs532_smoothed,sliced_range/1000.)
ax1.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax1.xaxis.major.formatter._useMathText = True
ax1.set_xlim(left = 0)
ax2.set(xlabel = r"$\frac{dLnRCS_{532}}{dz}$ [a.u.]")
ax2.plot(np.gradient(np.log(data['rcs_532xpa'].sel(time=slice(ini_date,end_date),range=slice(ini_range,end_range)).mean('time'))), sliced_range/1000., color = 'coral') #Plots the profile with the original (unsmoothed) products
ax2.plot(f_deriv_log_smoothed, sliced_range/1000.)
ax2.legend(('original', 'smoothed'), scatterpoints=1, loc='upper right', ncol=1, fontsize=10.5) #Represents the legend. Use only in case the original profile is represented
ax2.axvline(x = 0, color = "k")
if x_min is not None and x_max is not None:
    ax2.set(xlim = (x_min,x_max))
if y_min is not None and y_max is not None:
    ax2.set(ylim = (y_min,y_max))
ax2.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax2.xaxis.major.formatter._useMathText = True
for ax_ in [ax1, ax2]:
    font_axes(ax_,12)
    
#Optional. You can represent horizontal lines (as much as needed) to indicate the method height estimation for each layer. If 
#you decided not to use them, you can directly delete the lines or comment them (#__ o """__"""). To use them, simply write
#the height estimated value in height_n.

height_1 = 1. #km
ax1.axhline(y = height_1, color = "r")
ax2.axhline(y = height_1, color = "r")

plt.tight_layout()
plt.savefig(outputpath / f"dlnrcs_{ini_date}_{end_date}.png") 

**PERFIL DE VARIANZA DE SEÑAL CORREGIDA DE RANGO**

Finalmente, el siguiente módulo permitirá **realizar el método de la varianza de RCS**, que permite estimar las alturas de la capa límite atmosférica y capas desacopladas.

In [ ]:
%matplotlib notebook 
#------------------------------------------------------------------------------------------------------------------------------
#This four variables allow to zoom the graphic based on the previously observed profile. Some default values are given as a hint
x_min = None #Bottom value of the RCS variance in arbitrary units
x_max = None #Top value of the RCS variance in arbitrary units
y_min = None #Bottom value of the vertical range (height) in meters
y_max = None #Top value of the vertical range (height) in meters
#------------------------------------------------------------------------------------------------------------------------------

stddev = data['rcs_532xpa'].sel(time=slice(ini_date,end_date),range=slice(ini_range,end_range)).std('time')
stddev_smoothed = savgol_filter(stddev, points2smooth, 1)

#Plotting
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (11, 12), sharey = True)
fig.suptitle(f"\n\nGranada, {ini_date[6:8]}/{ini_date[4:6]}/{ini_date[0:4]}\n{ini_date[9:17]} - {end_date[9:17]} UTC")
ax1.set(xlabel = "RCS$_{532}$ [a.u.]", ylabel = "Range [km agl]")
ax1.plot(sliced_rcs532_smoothed,sliced_range/1000.)
ax1.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax1.xaxis.major.formatter._useMathText = True
ax1.set_xlim(left = 0)
ax2.set(xlabel = "Std. dev. [a.u.]")
ax2.plot(stddev, sliced_range/1000., color = 'coral') #Plots the profile with the original (unsmoothed) products
ax2.plot(stddev_smoothed, sliced_range/1000.)
ax2.legend(('original', 'smoothed'), scatterpoints=1, loc='upper right', ncol=1, fontsize=10.5) #Represents the legend. Use only in case the original profile is represented
if x_min is not None and x_max is not None:
    ax2.set(xlim = (x_min,x_max))
if y_min is not None and y_max is not None:
    ax2.set(ylim = (y_min,y_max))
ax2.ticklabel_format(axis="x", style="sci", scilimits=(0,0))
ax2.xaxis.major.formatter._useMathText = True
for ax_ in [ax1, ax2]:
    font_axes(ax_,12)

#Optional. You can represent horizontal lines (as much as needed) to indicate the method height estimation for each layer. If 
#you decided not to use them, you can directly delete the lines or comment them (#__ o """__"""). To use them, simply write
#the height estimated value in height_n.

height_1 = 0.6 #km
ax1.axhline(y = height_1, color = "r")
ax2.axhline(y = height_1, color = "r")

plt.tight_layout()
plt.savefig(outputpath / f"stddev_{ini_date}_{end_date}.png") 